<a href="https://colab.research.google.com/github/urvashi2004/ML_MiniProjects/blob/main/SkinCancer/OptimisedVGG16ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import drive, files

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
drive_base_path = '/content/drive/My Drive/Colab Notebooks/SkinCancer/Mendeley'
zip_file_path = os.path.join(drive_base_path, 'mendeleydataset.zip')
csv_file_path = os.path.join(drive_base_path, 'metadata.csv')
dataset_dir = '/content/dataset'

# Extract dataset if not already extracted
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir, exist_ok=True)
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)
    print(f"Dataset extracted to {dataset_dir}.")
else:
    print(f"Dataset already exists at {dataset_dir}.")

Mounted at /content/drive
Extracting dataset...
Dataset extracted to /content/dataset.


In [3]:
# Adjust dataset subdirectory if necessary
image_dir = os.path.join(dataset_dir, "mendeleydataset")  # Modify this if images are in another subdirectory
if not os.path.exists(image_dir):
    image_dir = dataset_dir  # Fallback to top-level directory if no subdirectory found

# Reload metadata
labels_df = pd.read_csv(csv_file_path)
labels_df['label'] = labels_df['biopsed'].apply(lambda x: 'cancer' if x else 'non_cancer')

# Map images to labels
image_paths, labels = [], []
for _, row in labels_df.iterrows():
    img_path = os.path.join(image_dir, row['img_id'])
    if os.path.exists(img_path):
        image_paths.append(img_path)
        labels.append(row['label'])

print(f"Found {len(image_paths)} images with labels.")

Found 2298 images with labels.


In [4]:
# Split dataset
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_paths, 'class': train_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': val_paths, 'class': val_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 1838 validated image filenames belonging to 2 classes.
Found 460 validated image filenames belonging to 2 classes.


In [5]:
# Define model function
def build_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train VGG16
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_model = build_model(vgg_base)
vgg_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights
)

# Train ResNet50
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model = build_model(resnet_base)
resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


58/58 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 0.5016 - loss: 0.7639 - val_accuracy: 0.5848 - val_loss: 0.6929
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.5722 - loss: 0.6922 - val_accuracy: 0.5848 - val_loss: 0.6931
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5724 - loss: 0.6940 - val_accuracy: 0.5848 - val_loss: 0.6931
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5891 - loss: 0.6919 - val_accuracy: 0.5848 - val_loss: 0.6931
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.5908 - loss: 0.6915 - val_accuracy: 0.5848 - val_loss: 0.6931
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.5477 - loss: 0.6959 - val_accuracy: 0.5848 - val_loss: 0.6931
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.5495 - loss: 0.6983 - val_accuracy: 0.5848 - val_loss: 0.6931
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.5755 - loss: 0.6936 - val_accuracy: 0.4152 - val_loss: 0.

In [10]:
# Save models
vgg_model.save('/content/vgg_model.h5')
resnet_model.save('/content/resnet_model.h5')

# Define preprocessing function for predictions
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return img_array

# Predict function
def predict_image(image_path, model):
    img = preprocess_image(image_path, target_size=(224, 224))
    img_array = np.expand_dims(img, axis=0)
    prediction = model.predict(img_array)
    return 'Cancerous' if prediction[0][0] > 0.5 else 'Non-Cancerous'

# Upload and test an image
print("Upload an image to test the model.")
uploaded = files.upload()
uploaded_image_path = list(uploaded.keys())[0]

print(f"Prediction using VGG16: {predict_image(uploaded_image_path, vgg_model)}")
print(f"Prediction using ResNet50: {predict_image(uploaded_image_path, resnet_model)}")

Upload an image to test the model.


Saving PAT_522_3168_747.png to PAT_522_3168_747.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Prediction using VGG16: Cancerous
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction using ResNet50: Cancerous
